This notebook includes the code builds a matrix between the hexagonal cells (or any array of coordinate pairs). The input is the output of the data preprocessing notebook (I think in your repository the file "felix_data.csv"). However, it will be easy to modify the code slighty in order to calculate distances between other coordinates if that would be needed in the future.
For information on the routing, see http://project-osrm.org/docs/v5.24.0/api/#

In [ ]:
import requests
import pandas as pd 
import json
import numpy as np
import itertools


In [ ]:
# first, we need the file.
df = pd.read_csv("test_out.csv") # insert correct path and name here. should be the output of the other notebook.
df.columns = ['nr', 'id', 'delta', 'x_coord', 'y_coord'] # can be modified/deleted if another input is used
# some sanity checks (coords far away from Munich will be dropped)
df = df[(47.5<df.x_coord) & (df.x_coord<48.5) & (11<df.y_coord) & (df.y_coord<12)]
# build a dictionary with coordinates as tuple and id as key; first build a row with coords as a tuple
df['lat_lon_tuple'] = list(zip(df.y_coord,df.x_coord))
coords = pd.Series(df.lat_lon_tuple.values,index=df.id).to_dict()

In [ ]:
# a query can't contain 743*743 coordinate tuples. we therefore have to slice it into chunks, which we will later on reassemble.
# as we always combine to "packs" (origins and destinations), the list for the query will contain 2*max_size coordinate tuples.
# queries containing 100 tuples have shown to run stable.
max_size = 50
coord_packs = []
for i in range (0, len(coords)//max_size+1,1):
    slice_start = i*max_size
    slice_end = slice_start + max_size
    coord_packs.append(dict(itertools.islice(coords.items(), slice_start, slice_end)))
#coord_packs.append(dict(itertools.islice(coords.items(), (len(coords)-len(coords)%max_size), len(coords)))) # used in a former version to add the last, shorter bunch
#coord_packs = coord_packs[13:] # use this for debugging to limit the size of the matrix


In [ ]:
# a query has the following form:
# /table/v1/{profile}/{coordinates}?{sources}=[{elem}...];&{destinations}=[{elem}...]&annotations={duration|distance|duration,distance}
# in this cell, we procuce the {sources} and {destinations}, telling OSRM how to interpret the list of coordinates

def create_mapping(origin_len, dest_len):
    sources = ""
    destinations = ""
    for i in range (0, dest_len, 1):
        destinations = destinations + (str(i+origin_len)+';')
    for i in range (0, origin_len, 1):
        sources = sources + (str(i)+';')
    sources = sources[:-1]
    destinations = destinations[:-1]
    return sources, destinations


In [ ]:
# now we query the sub-matrices and assemble them to a big one. On my machine (FYI: i710850H 2,7 GHz 16GB RAM) this takes  ~23 min.
res = pd.DataFrame()
for col, base_coords in enumerate(coord_packs):
    res_col_tmp = pd.DataFrame()
    for row, target_coords in enumerate(coord_packs):
        print(f"col: {col}\t\trow: {row}") # can be disabled if annoying (get's pretty long)
        # make a string from coordinate tuples
        coords_origin = "".join([str(val[0])+','+str(val[1])+';' for val in list(base_coords.values())])
        coords_destination = "".join([str(val[0])+','+str(val[1])+';' for val in list(target_coords.values())])
        # add origins and destinations, delete the last ';'
        coords_str = (coords_origin+coords_destination)[:-1]
        sources, destinations = create_mapping(len(base_coords), len(target_coords))
        req_str=f"http://router.project-osrm.org/table/v1/driving/{coords_str}?sources={sources}&destinations={destinations}&annotations=distance"
        # get the request
        r = requests.get(req_str)
        # load the response as a json and retrieve the relevant information. Unit: meters
        durs_tmp = json.loads(r.content).get("distances")
        # load list of lists into DF
        res_row_tmp = pd.DataFrame(durs_tmp, index=list(base_coords.keys()), columns=list(target_coords.keys()))
        # assemble the column in the temporary DF
        res_col_tmp = pd.concat([res_col_tmp, res_row_tmp],axis=1)#res.append(res_tmp)
        # clean the variables just to be sure
        coords_str, coords_origin, coords_destination = "", "", ""
    # add the complete column to the results DF
    res = pd.concat([res, res_col_tmp], axis=0)#res.append(res_tmp)

res

In [ ]:
# save the file as pickle and csv
res.to_pickle("adjacency_matrix.pkl")
res.to_csv("adjacency_matrix.csv")

In [ ]:
# this is another version based on iterating over each cell - you will notice its too slow (even for my standards ;)
# we need a DataFrame to store the results in
res = pd.DataFrame()
# now we iterate over the coordinates. 
count = 0 # counter for storing
for base_id, base_coord in coords.items():
    # remove the current coordinate so we can check the ways to all others
    tmp_coords = coords.copy()
    del tmp_coords[base_id]
    print(count)
    tmp_adjacencies = []
    # now for each coord, we need to find the way to all other coords
    for target_id, target_coord in tmp_coords.items():
        count += 1
        # get the route from OSMR: http://project-osrm.org/docs/v5.7.0/api/?language=Python#general-options
        r = requests.get(f"http://router.project-osrm.org/route/v1/driving/{base_coord[1]},{base_coord[0]};{target_coord[1]},{target_coord[0]}?overview=false&steps=false&overview=false")
        # uncomment the following lines for debugging or if interested
        #dist = json.loads(r.content).get("routes")[0].get('distance')
        #print(json.loads(r.content))
        #print(f" start: {base_coord}, end: {target_coord}, dist: {dist}")
        res.loc[base_id, target_id] = json.loads(r.content).get("routes")[0].get('distance')
        # this takes long, so we should store from time to time.
        if count % 1000 == 0:
            res.to_pickle("res_df.pkl")